<a href="https://colab.research.google.com/github/higorpr/ENEM_Exploration_Analysis/blob/main/ENEM_Exploratory_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Methodology

Hypothesis > Mathematical/Statistical Theory > Artifact > Experiment > Hypothesis Update and New Iteration

# Environment Setup

Installing libraries

In [36]:
!pip install -U -q PyDrive

ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


Importing libraries

In [37]:
import pprint
import io
import pandas as pd

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import auth
from googleapiclient.discovery import build

from googleapiclient.http import MediaIoBaseDownload

Getting train and test files from Google Drive:

In [2]:
#@title uploader

file_id = "1-HDq18PMcjLkRnbcLb36kA8dAHy5V55q" #@param {type:"string"}

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

auth.authenticate_user()

drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

data.zip
Archive:  data.zip
  inflating: test.parquet            
  inflating: train.parquet           


Importing train and test files as Pandas Dataframes:

In [3]:
df_test = pd.read_parquet('test.parquet', engine='pyarrow')
df_train = pd.read_parquet('train.parquet', engine='pyarrow')

Target features to be predicted in a future ML model:

In [4]:
set(df_train.columns) - set(df_test.columns)

{'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'}

Columns in the training dataset (features and targets):

In [4]:
for col in df_train.columns:
    print(col)

NU_INSCRICAO
CO_MUNICIPIO_RESIDENCIA
NO_MUNICIPIO_RESIDENCIA
CO_UF_RESIDENCIA
SG_UF_RESIDENCIA
NU_IDADE
TP_SEXO
TP_ESTADO_CIVIL
TP_COR_RACA
TP_NACIONALIDADE
CO_MUNICIPIO_NASCIMENTO
NO_MUNICIPIO_NASCIMENTO
CO_UF_NASCIMENTO
SG_UF_NASCIMENTO
TP_ST_CONCLUSAO
TP_ANO_CONCLUIU
TP_ESCOLA
TP_ENSINO
IN_TREINEIRO
CO_ESCOLA
CO_MUNICIPIO_ESC
NO_MUNICIPIO_ESC
CO_UF_ESC
SG_UF_ESC
TP_DEPENDENCIA_ADM_ESC
TP_LOCALIZACAO_ESC
TP_SIT_FUNC_ESC
IN_BAIXA_VISAO
IN_CEGUEIRA
IN_SURDEZ
IN_DEFICIENCIA_AUDITIVA
IN_SURDO_CEGUEIRA
IN_DEFICIENCIA_FISICA
IN_DEFICIENCIA_MENTAL
IN_DEFICIT_ATENCAO
IN_DISLEXIA
IN_DISCALCULIA
IN_AUTISMO
IN_VISAO_MONOCULAR
IN_OUTRA_DEF
IN_GESTANTE
IN_LACTANTE
IN_IDOSO
IN_ESTUDA_CLASSE_HOSPITALAR
IN_SEM_RECURSO
IN_BRAILLE
IN_AMPLIADA_24
IN_AMPLIADA_18
IN_LEDOR
IN_ACESSO
IN_TRANSCRICAO
IN_LIBRAS
IN_TEMPO_ADICIONAL
IN_LEITURA_LABIAL
IN_MESA_CADEIRA_RODAS
IN_MESA_CADEIRA_SEPARADA
IN_APOIO_PERNA
IN_GUIA_INTERPRETE
IN_COMPUTADOR
IN_CADEIRA_ESPECIAL
IN_CADEIRA_CANHOTO
IN_CADEIRA_ACOLCHOADA
IN_PROVA

# Exploratory Analysis

In this section, we will perform an exploratory analysis based on the main divisions on the data:

Participant Personal Information Features (PPI):

In [46]:
# Creating list with PPI features:
tag_col = 'NU_INSCRICAO'
first_index = df_train.columns.get_loc('CO_MUNICIPIO_RESIDENCIA')
last_index = df_train.columns.get_loc('IN_TREINEIRO')
ppi_index = list(range(first_index, last_index))
ppi_index.append(df_train.columns.get_loc(tag_col))
ppi_index = sorted(ppi_index)
ppi = list(df_train.columns[ppi_index])

# Printing PPI features:
pp = pprint.PrettyPrinter(indent=4)
print("The features related to a student's personal information are:\n")
pp.pprint(ppi)

The features related to a student's personal information are:

[   'NU_INSCRICAO',
    'CO_MUNICIPIO_RESIDENCIA',
    'NO_MUNICIPIO_RESIDENCIA',
    'CO_UF_RESIDENCIA',
    'SG_UF_RESIDENCIA',
    'NU_IDADE',
    'TP_SEXO',
    'TP_ESTADO_CIVIL',
    'TP_COR_RACA',
    'TP_NACIONALIDADE',
    'CO_MUNICIPIO_NASCIMENTO',
    'NO_MUNICIPIO_NASCIMENTO',
    'CO_UF_NASCIMENTO',
    'SG_UF_NASCIMENTO',
    'TP_ST_CONCLUSAO',
    'TP_ANO_CONCLUIU',
    'TP_ESCOLA',
    'TP_ENSINO']


RangeIndex(start=0, stop=3311925, step=1)